<a href="https://colab.research.google.com/github/emerenan/xlsx_validation/blob/main/ro_validations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [110]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import re
from openpyxl import Workbook


def read_files(path, sheetname, n_skiprows, n_skip_columns, site_index):
    """
    Params:\n
    path: parth of file in the computer.\n
    n_skiprows: Number of rows to delete in the original file,.\n
    columns_to_convert: Columns to convert the data general type. \n
    n_skipcolumn: Columns to skip in the original file. \n
    endrow = pass 0 to read everything, 1 to count entire
    columns_order: List of columns names in specific order to pass in the engine.\n
    """
    df = pd.read_excel(path, sheet_name = sheetname, skiprows = n_skiprows)

    df = df.iloc[:,n_skip_columns:]

    # define the entiry columns which doesn't have NaN 
    df = df.dropna(subset=[site_index], axis=0)
    """cont = 0
    for i in df.iloc[:,site_col]:
        if pd.isnull(i):
            break # acertar o break
        else:
            cont +=1
    df = df.iloc[:cont, :]
    #df.columns = columns_order"""
    
    # convert intery columns to integer 
    #df[columns_integer_convert] = df[columns_integer_convert].fillna(0)
    #df[columns_integer_convert] = df[columns_integer_convert].astype('int64')

    return df

def check_df(df, error_msg):
    if df == None or df.empty:
        return f'{error_msg}'
    else:
        return df

def count_duplicates(lista):
    count_dict = {}
    for entry in lista:
        if entry in count_dict.keys():
            count_dict[entry] += 1
        else:
            count_dict[entry] = 1
    
    duplicates = {}
    for k, v in count_dict.items():
        if v > 1:
            duplicates[k] = v
    return pd.DataFrame.from_dict(duplicates, orient='index', columns=['# of Duplicates'])

def defining_df(df, column_range, number_col):
    cont = 0
    for i in df.iloc[:,number_col]:
        if pd.isnull(i):
            break # acertar o break
        else:
            cont +=1
    df = df.iloc[0:cont, :]
    return df

#old version
def check_columns(table, output_columns):
    """
    Check the total of number of missing columns and the missing columns in passed table.\n

    Params:\n
    table: contain the columns to be check.\n
    output_columns: columns structure at the final file. 

    Returns:\n
    Number of missing columns and a list that contains the name os missing columns.
    """
    """"
    countries = ['DE':{'towerDB':[],\
                       'lc': [],\
                       'ta':[]}\,
                 'HU',
                 'IE',
                 'RO',
                 'PT',
                 'ES',
                 'CZ': {'towerDB':["Code (Duplicate)","Site Status","VF - In scope / out of scope (Generalised scoping)","Site in Skylon scope Actual (From Site List Sheet )","Legacy Site Code(Duplicate)","TIMS Site Code","Legacy Site Code","Site Name","Macro Region","Province","Municipality","Inhabitants","Address","Ground Register","Altitude","Latitude","Longitude","Categorization by Inhabitants","Categorization by Transmission Sys","Categorization by Site Type","Categorization by Transmission Sys (subcluster)","Other internal Categorization 1 (Identify ACQ Sites)","Other internal Categorization 2 Energy provider (Eon/ LL)","DAS+Macro","DAS (Yes/ No)","DAS Ownership (Complete/ Partial/ 3rd Party)","Active/ Passive DAS","# of remote units/ radiating points","Type of Structure","Distance highest antenna to ground level","GBT Tower height","POD ID","Energy Consumption LTM (kwh)","Annual Energy cost LTM (Euros)","Infrastructure ready (existing)/ to be ready (new)","Infrastructure to be dismantled by","Radio equipments to be deactivated by","Infrastructure to be shared by","Technology VOD","Fibre / Microwave","Vertical passive structure owner","Room configuration (detailed)","Shelter passive structure ownership","Type of Air Conditioning","Number of cabinets (Full Capacity)","Number of Antenna (Full Capacity)","Number of MW (Full Capacity)","Counterpart","# of Lease Contracts","Current annual lease fees ","Current other fees (Maintenance)","Current other fees","(Average) residual duration - Lease contract","(Average) residual duration - Maintenance contract","(Average) residual duration - Lease & Maintenance both","# of Tenants Agreements","Current Total Annual Hosting Fees","Tenant (name/ID) MNO1 (Česká telekomunikační infrastruktura a.s.)","Annual Fee per Tenant MNO1","Annual Energy Fee MNO1","Annual Maintenance Fee MNO1","Other Services Fee MNO1","Residual duration MNO1 (Years)","Tenant (name/ID) MNO2 (T-Mobile Czech Republic a.s.)","Annual Fee per Tenant MNO2","Annual Energy Fee MNO2","Annual Maintenance Fee MNO2","Other Services Fee MNO2","Residual duration MNO2 (days)","Tenant (name/ID) MNO3","Annual Fee per Tenant MNO3","Annual Energy Fee MNO3","Annual Maintenance Fee MNO3","Other Services Fee MNO3","Residual duration MNO3","# of OTMOs","Annual Fee from OTMOs","Annual Energy Fee from OTMOs","Annual Maintenance Fee OTMOs","Other Services Fee OTMOs","Average residual duration (days)","Check","Strategic Macro Sites","Critical Sites","Case A Core Site","Macro Site - Transmission Hub Site","Macro Site - Transmission Hub Site with/without Shelters","Transmission Sites","Room Configuration","Power Supply","Air Conditioning","Active Sharing Arrangements involving the Operator","VF-CZ Demerger phase","EVO Location [FAR Site ID] ","Billing Trigger date ",\
                 "Strategic_Site_Bucket","Critical Site Bucket","Subsequent_Sharing_Arrangement","First_Active_Sharing_Deployment_Type","First_Active_Sharing_Start_Date","First_Active_Sharing_End_Date","Decommissioned_Site","Wip_Site","Bts_Site","Transfer_Date_Of_Consent_Required_Sites","Sites_As_Metered_Estimated","Date_Of_Equipment_Removal"],\
                       'lc': [],\
                       'ta':['Tenant Agreement ID - NEW', 'Code', 'Site Name', 'Service Type','Contract start date', 'Contract end date', 'Status of contract','Renewal Option', 'Comments', 'Counterpart ID', 'Counterpart','Classification of Tenant', 'Annual amount - billing currency','Billing Currency', 'Annual Amount in CZK', 'Amount in EUR','Terms of Payment', 'Frequency', 'Indexed YES/NO', 'Index','Percentage', 'VAT Subject YES/NO', 'Percentage (VAT)', 'Unique Key','Classification', 'Residual Period in Years', 'Remarks','Count of unique key', 'Count of contracts', 'Scoping classification','DAS sites', 'DAS ownership', '31-Mar-21', 'Update in month','Updated Item', 'Comment', 'Counterpart_extra_1', 'Counterpart_extra_2', 'x','SiteType', '1.028', 'Unique Tenant', 'Unique Tenant Count', 'not_def_1']}\,
                 }
                 'GR']
                 """
    total_received = len(table.columns)
    number_missing_columns = 0
    missing_columns = []
    #Counting of missing columns       
    #if country in contries:      
    for columns in output_columns:
        if columns.lower() not in [labels.lower() for labels in table]:
            number_missing_columns +=1
            missing_columns.append(columns)
    
    return total_received, number_missing_columns, missing_columns

def check_columns_received(df, bill_cols):
    twdb_col = [i for i in df.columns]
    col_miss = [ i for i in bill_cols if i not in twdb_col]
    for i in bill_cols:
        if i not in twdb_col:
            col_miss.append(i)
    df_col_missing = pd.DataFrame(col_miss, columns=['Column(s) Missing'], index=range(len(col_miss)))
    return df_col_missing

def replace_values(df, columns, value=""):
    """
    Está voltando para float
    """
    invalid_values = ['N/A', 'n/a', 0, '-', '_', np.nan,'nan']
    #lista = []

    df.fillna(0, inplace=True)
    #df[column] = df[column].astype('int64')

    for column in columns:
        lista = []
        for index in df[column]:
            #print(f"{column} -> {index}")
            if index in invalid_values:
                lista.append(value)
            else:
                lista.append(index)
        df[column] = lista

    return df
      
def date_parser(df, columns, format, type_dates):
    t_col = type_dates.lower()
    for column in columns:
        if t_col == 'mixed':
            df[column] = [date_obj.strftime(format) if not pd.isnull(date_obj) \
                        and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
            df[column] = df[column].astype(str)
        else:
            df[column] = [date_obj.strftime(format) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
            df[column] = df[column].astype(str)

# Refactorar esse codigo para receber todas as colunas num dic
# Sendo as keys=columns e values= picklist for each column
def check_date_columns(df, df_index, columns, format):
    """
    Paramns \n
        Dates needs to be in string format not in datetime, otherwise raise an error.\n
        Convert entire column to string before.
    """
    df_dates = df[columns]
    df_dates['sites'] = df_dates[df_index]
    df_dates = df_dates.set_index('sites')

    df_de = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df_dates[df_index])
    if not df_duplicates.empty:
        df_dates.drop_duplicates(subset=[df_index], inplace=True)
    
    if format == 1:
        date_format = re.compile(r'\d{2}\-\d{2}\-\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                #print(type(df_dates.loc[de_site,de_column]))
                if date_format.match(df_dates.loc[de_site,de_column]) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        df_de.loc[de_site,de_column] = 'Incorret Format or Blank Value'
                    else:
                        df_de.loc[de_site,de_column] = 'Incorret Format or Blank Value'
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            return df_de
        else: 
            print('No one columns with incorrect date format!')
    else:
        date_format = re.compile(r'\d{2}\/\d{2}\/\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                if date_format.match(df_dates.loc[de_site,de_column]) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        df_de.loc[de_site,de_column] = 'Incorret Format or Blank Value'
                    else:
                        df_de.loc[de_site,de_column] = 'Incorret Format or Blank Value'
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            return df_de
        else: 
            print('No one columns with incorrect date format!')

def check_amounts(df_check, df_index, columns, pattern):
    """
    Paramns:
    pattern: general (. to decimal), other (, to decimal)
    """

    df = df_check[columns]
    df['sites'] = df[df_index]
    df = df.set_index('sites')

    df_new = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df[df_index])
    if not df_duplicates.empty:
        df.drop_duplicates(subset=[df_index], inplace=True)

    for site in df[df_index]:
        for column in columns[1:]:
            #print(f'{df_dates.loc[de_site,df_index]}: {df_dates.loc[de_site,de_column]}')
            if pattern == 1:
                if not str(df.loc[site,column]).__contains__('.'):
                    #print(str(df.loc[site,column]))
                    if df.loc[site,df_index] not in df_new.index:
                        df_new.loc[site,df_index] = df.loc[site,df_index]
                        df_new.loc[site,column] = 'Incorret Format to separate decimal values'
                    else:
                        df_new.loc[site,column] = 'Incorret Format to separate decimal values'
            else:
                if not str(df.loc[site,column]).__contains__(','):
                    #print(str(df.loc[site,column]))
                    if df.loc[site,df_index] not in df_new.index:
                        df_new.loc[site,df_index] = df.loc[site,df_index]
                        df_new.loc[site,column] = 'Incorret Format to separate decimal values'
                    else:
                        df_new.loc[site,column] = 'Incorret Format to separate decimal values'


    df_new = df_new.dropna(how='all', axis=1).fillna('Ok')       
    if not df_new.empty:
        return df_new
    else: 
        print('No one columns with incorrect Amount format!')
        
def check_picklist(df,df_index,df_cols, picklist_dict):
    """
    Params:
        df:
        df_index: 
        picklist_dict:
        nultiIndex: More than one column to check
    """
    df_picklist = df[df_cols]
    df_picklist['sites'] = df[df_index]
    df_picklist =  df_picklist.set_index('sites')
    
    #df_picklist = replace_values(df_picklist, df_cols, 0)

    df_errors = pd.DataFrame(columns=df_cols)

    for site in df[df_index]:
        for column in picklist_dict.keys(): 
            value = str(df_picklist.loc[site,column])
            col_values = [i.lower() for i in picklist_dict[column]]
            if not value.lower() in col_values or pd.isnull(value):
                if not df_picklist.loc[site,df_index] in df_errors.index:
                    df_errors.loc[site,df_index] = df_picklist.loc[site,df_index]
                    df_errors.loc[site,column] = 'Incorret picklist value or Blank Value'
                else:
                    df_errors.loc[site,column] = 'Incorret picklist value or Blank Value'
    #df = df_errors.dropna()   
    df = df_errors.dropna(how='all', axis=1)   
    return df

def check_picklist_3(df,df_index, picklist_dict):
    log = {}
    for column in picklist_dict:
        df_aux = df.copy()
        new = df_aux[column].isin(picklist_dict[column])
        #new = df_aux[column].apply(lambda x: x in picklist_dict[column])
        # Aceita somento os valores que não estão na picklist
        indexes = df.index[new == False].tolist()
        #if len(indexes)>0:
        if column not in log:log[column]=[]
        log[column]=log[column]+indexes
    #print(log.keys())

    newDict ={}
    df1 = pd.DataFrame()
    for key,value in log.items():
        for val in value:  
            ID=df.iloc[val][df_index]
            if ID in newDict:
                newDict[ID].append(key)
            else:
                newDict[ID] = [key]
        
    logs = pd.DataFrame.from_dict(newDict, orient='index')
    return logs

def check_on_air_sites(df, df_index, df_cols):

    df_check = df[df_cols]
    df_check['sites'] = df_check[df_index]
    df_check = df_check.set_index('sites')
    #df_check = replace_values(df_check, df_cols, 0)

    df_errors = pd.DataFrame(columns=df_cols)

    for site in df[df_index]:
        for column in df_cols: 
            if df_check.loc[site,df_index] not in df_errors.index:
                df_errors.loc[site,df_index] = df_check.loc[site,df_index]
                if df_check.loc[site,column] == 0:
                    df_errors.loc[site,column] = 'Incorret picklist value or Blank Value'
            else:
                if df_check.loc[site,column] == 0:
                    df_errors.loc[site,column] = 'Incorret picklist value or Blank Value'
    df = df_errors[df_errors.iloc[:,1:]]
    df = df.dropna(how='all', axis=1).fillna('Ok')       
    #df = df_errors.dropna(how='all', axis=1)   
    return df

def check_new_sites(df_towerdb, tw_index, bts_col, bill_col, msa_list, towerdb_list, uip_list):
    
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    
    #Finding for ALL NEW SITES
    new_site = [str(i) for i in towerdb_list if i not in msa_list]
    new_sites = pd.DataFrame(new_site, columns=['New_Sites'])
    
    #list of new sites out of UIP sheet
    bts_out_uip = [str(i) for i in df_towerdb[df_towerdb[bts_col]=='Yes'][tw_index].sort_values() if i not in uip_list]
    bts_out_uip = pd.DataFrame(bts_out_uip, columns=['Bts_Sites_Out_UIP_File'])

    #create a copy to make index modifications
    df = df_towerdb.copy()

    #New columns with Site Codes
    df['Sites'] = df[tw_index]
    #defining site code to index
    df.set_index('Sites', inplace=True)

    #filtering by new sites
    df = df[df[tw_index].isin(new_site)]

    # Save information os sites with demerged date more than current date
    df[bill_col] = df[bill_col].astype('datetime64[s]')
    df_site_bts = df[(df[bts_col]=='Yes') | (df[bill_col] > current_date)]

    return new_sites, bts_out_uip, df_site_bts[[tw_index, bts_col, bill_col]]
    """Restruturar o script em CZ, DE, PT"""

def check_bts(df_tw, bts_tw_columns, tw_index, df_msa, bts_msa_column, msa_index):
    #Nested Function to make conditional validations
    def cond_bts_check(bts_msa, tw_bts_sites):
        bts_out_tw=[]
        if sorted(bts_msa) != sorted(tw_bts_sites):
            for i in tw_bts_sites:
                if i not in bts_msa:
                    bts_out_tw.append(i)

        return bts_out_tw

    bts_msa = msa[msa[bts_msa_column]=='Yes']
    bts_msa = [str(i) for i in bts_msa[msa_index]]

    tw_bts_sites = df_tw[df_tw[bts_tw_columns]=='Yes']
    tw_bts_sites = [str(i) for i in tw_bts_sites[tw_index]]

    #return of datas
    bts_out_tw = cond_bts_check(bts_msa, tw_bts_sites)
    df = pd.DataFrame(bts_out_tw, columns=['New Sites'])
    return df

def check_wip(df_tw,tw_index, wip_tw, tw_bts, df_msa, msa_index, wip_msa_col):

    #Nested Function to make conditional validations
    def cond_wip_check(wip_msa, tw_wip_sites):
        count_wip = 0
        wip_out_tw=[]
        if sorted(wip_msa) != sorted(tw_wip_sites):
            for i in tw_wip_sites:
                if i not in wip_msa:
                    count_wip += 1
                    wip_out_tw.append(i)

        return wip_out_tw

    wip_msa = df_msa[df_msa[wip_msa_col]=='Yes']
    wip_msa = [str(i) for i in wip_msa[msa_index]]

    tw_wip_sites = df_tw[df_tw[wip_tw]=='Yes']
    tw_wip_sites = [str(i) for i in tw_wip_sites[tw_index]]

    tw_wip_site_bts_flagged = df_tw[(df_tw[wip_tw]=='Yes')&(df_tw[tw_bts]=='Yes')]
    tw_wip_site_bts_flagged = tw_wip_site_bts_flagged[[tw_index, wip_tw, tw_bts]]

    wip_out_tw_list = cond_wip_check(wip_msa, tw_wip_sites)
    return wip_out_tw_list, tw_wip_site_bts_flagged
    """Reestrurar os script em PT, DE, CZ"""
    # Falta os outros países

def check_decommissioned(df,df_index, decom_col, doer_col):
    #c = country.lower()
    filtered = df[(df[decom_col]=='Yes')&(df[doer_col]=="")]
    return filtered[[df_index, decom_col, doer_col]]
    """Ajustar para CZ, DE, PT"""
    
def check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col):
    
    t = type_col.lower()
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    if not df_tw[date_col].empty:
        if t == 'doer':
            filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col].astype('datetime64[ns]') < current_date)]
            return filtered[[tw_index, status_col, date_col]] 
        else:
            filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col].astype('datetime64[ns]').empty)]
            return filtered[[tw_index, status_col, date_col]] 
    else:
        print('Nothing wrong in services sites!')
    """Ajustar para CZ, DE, PT, IE"""

def check_tw_doer_planned(df_tw, tw_index, doer_col, status_col):
    """Only GR until now"""
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    if not df_tw[doer_col].empty:
        filtered = df_tw[(df_tw[status_col]=='Planned')&(not df_tw[doer_col].astype('datetime64[ns]').empty)]
        return filtered[[tw_index, status_col, doer_col]]  
    else:
        print('Nothing wrong in services sites!')
                                                              
def check_mom_bts(df_tw, tw_index, tw_col, df_msa,msa_index, msa_col):

    #c = country   
    msa_bts = df_msa[df_msa[msa_col]=='Yes']
    msa_bts_sites = [i for i in msa_bts[msa_index]]

    tw_bts = df_tw[df_tw[tw_col]=='Yes']
    tw_bts_sites = [i for i in tw_bts[tw_index]]

    out_tower_bts = [i for i in msa_bts_sites if i not in tw_bts_sites]
    filtered = tw_bts[tw_bts[tw_index].isin(out_tower_bts)]
    return filtered[[tw_index, tw_col]]         

def check_lc_ta_dates(df,tw_index, start_date,end_date):

        filtered = df[df[start_date] > df[end_date]]
        return filtered[[tw_index, start_date,end_date]]

def check_uip_tw(df_tw,tw_index, status_tw_col, decom_col, tw_bts_col, tw_critical_col, df_uip, uip_sites, country):
    t1 = ['pt', 'de', 'cz', 'ie', 'es', 'ro']
    if country.lower() in t1:
        #tw_active = df_tw[df_tw['Site Status']=='In Service']
        count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

        # check number of sites that are in uip file and doesn't have in df_tw
        in_service_uip_sites = []
        if not set(count_tw_sites).intersection(uip_sites):
            in_service_uip_sites = [i for i in uip_sites if i not in count_tw_sites]
        in_service_uip_sites = pd.DataFrame(in_service_uip_sites,columns=['Site In service out of UIP File!'])
        
        #check for decomissioned site not in uip files
        tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index] if i in uip_sites]
        decomiss_sites_in_uip = pd.DataFrame(tw_decomiss, columns=['Decomissioned Site in UIP File'])
        
        #Check BTS sites
        bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
        bts_sites_out_uip = []
        if not set(bts_sites).intersection(uip_sites):
            bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
        bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIP File'])

        #  Check for UIP critical sites 
        uip_critical = [i for i in df_uip[(df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='')&\
                                    df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]
        bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
        critical = []
        if len(uip_critical) > 0:
            if set(uip_critical).intersection(bts_tw_critical):
                critical = [i for i in bts_tw_critical if i not in uip_critical]
        critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIP File'])

        return in_service_uip_sites, decomiss_sites_in_uip, bts_sites_out_uip, critical
    else:
        #tw_active = df_tw[df_tw['Site Status']=='In Service']
        count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

        # check number of sites that are in uip file and doesn't have in df_tw
        in_service_uip_sites = []
        if not set(count_tw_sites).intersection(uip_sites):
            in_service_uip_sites = [i for i in uip_sites if i not in count_tw_sites]
        in_service_uip_sites = pd.DataFrame(in_service_uip_sites,columns=['Site In service out of UIP File!'])
        
        #check for decomissioned site not in uip files
        tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index]]
        decomiss_sites_in_uip = []
        if set(tw_decomiss).intersection(uip_sites):
            decomiss_sites_in_uip = [i for i in tw_decomiss if i in uip_sites]
        decomiss_sites_in_uip = pd.DataFrame(decomiss_sites_in_uip, columns=['Decomissioned Site in UIP File'])
        
        #Check BTS sites
        bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
        bts_sites_out_uip = []
        if not set(bts_sites).intersection(uip_sites):
            bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
        bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIP File'])

        #  Check for UIP critical sites 
        uip = [i for i in df_uip['Site_ID']]
        bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Yes'][tw_index]
        critical = []
        if set(uip).intersection(bts_tw_critical):
            critical = [i for i in bts_tw_critical if i not in uip]
        critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% out in UIP File'])

        return in_service_uip_sites, decomiss_sites_in_uip, bts_sites_out_uip, critical

def check_commercial(path_current, path_before, col_replace, col_names, merge_cols, col_order):
    
    def check_uip_commercial_values(df_actual, df_before, merge_cols):
        df_atual = uip_comercial_actual
        df_ant = uip_comercial_before
        df_cross = pd.merge(df_actual, df_before, on=merge_cols, \
                            how='left', suffixes=('_actual', '_before'), indicator='Exist')
        df_cross['Exist'] = np.where(df_cross.Exist == 'both', 'Yes', 'No')
        df_cross['Equal Values'] = df_cross['Exist']
        
        return df_cross
    # Check for commercial Values into current UIP File and compare with UIP File before
    uip_comercial_actual = pd.read_excel(path_current,sheet_name='Commercial', names=col_names)
    #uip_comercial_actual = uip_comercial_actual[['Charge_Type', 'Data_Type', 'Input_Value']]
    #uip_comercial_actual = replace_values(uip_comercial_actual, col_replace, value=0)

    uip_comercial_before = pd.read_excel(path_before,sheet_name='Commercial', names=col_names )
    #uip_comercial_before = uip_comercial_before[['Charge_Type', 'Data_Type', 'Input_Value']]
    #uip_comercial_before = replace_values(uip_comercial_before, col_replace, value=0)

    df_commercial =  check_uip_commercial_values(uip_comercial_actual, uip_comercial_before, merge_cols)

    df_commercial = df_commercial.reindex(columns=cols_ordered)
    df_commercial_diffs = df_commercial[df_commercial['Equal Values']=='No']
    return df_commercial_diffs

def general_log_erros(df_list, sheet_list, path):
    writer = pd.ExcelWriter(path,engine='openpyxl')   
    for dataframe, sheet in zip(df_list, sheet_list):
        dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0)   
    writer.save() 

In [ ]:
towerdb.head()

In [10]:
path_tw = '/content/TowerDB_Romania_20210731.xlsx'
sheet = 'Enhanced towerDB'

towerdb = read_files(path_tw, sheet, 0, 0, 'Code')

col_order = ['Scope','Phase _1/_2','Code','Site Name','Macro Region','Region','Province','Municipality','Inhabitants','Address',\
             'Altitude','Latitude','Longitude','Categorization by Transmission Sys','Active or passive DAS',\
             'Unused sites (TowerCo holds the property rights but does not host any Operator Equipment or Other Customer equipment )',\
             'Non-Vodafone equipment Sites (TowerCo holds the property rights and does not host any Operator Equipment but hosts Other Customer equipment  )',\
             'PowerOff Sites','Decommissioned sites','Categorization by Transmission Sys (sub-cluster)','Core Type',\
             'Macro Site - Transmission Hub Site with/Transmission Hub Site without Shelters','Transmission sites – with/Transmission Site without shelters.','Room Configuration',\
             'Power Supply','Air Conditioning','Active Sharing Arrangements involving the Operator','Transmission Hub sites’','Antenna Positions','Weight of RRUs',\
             'Total power transmitted per site ','Categorization by Transmission Sys_1','Categorization by Site Type','Categorization by inhabitants',\
             'Rural/ Suburban/ Urban','No_1','Technology VOD','Fiber / Microwave','Type of Structure','Tower Height','Floor space','POD ID','Energy Provider',\
             'Energy cost FYTD_DEC_20','Energy consumption FYTD_DEC_20','Infrastructure ready (existing)/ to be ready (new)','Billing Trigger Date','No_2',\
             'Radio equipments to be deactivated by','Infrastructure to be shared by','Counterpart','# of Lease Contracts','Current annual lease fees  ',\
             'Current energy lease fees ','Current annual other fees ','Total Annual Lease','Sub-lease','(Average) residual duration','Maturity Cluster',\
             'ExCo rep. Avg Annual Lease costs','Total Energy Cost (Energy provider + LL)','VOD (y/n)','TLK','Annual Hosting TLK','Annual Hosting TLK (with Discount)',\
             'Annual Energy TLK','Annual Maintenance Fee TLK','Other Services Fee TLK','Total Revenues TLK','Total Revenues TLK (with Discount)','Residual duration TLK',\
             'Maturity Clusters TLK','Orange Total (without active sharing)','Orange(Transmission)','Orange(Indoor)','Orange Passive Shared Sites ',\
             'Annual Hosting ORANGE (Excl. active sharing)','Annual Hosting ORANGE (Excl. active sharing) (with Discount)',\
             'Annual Energy ORANGE(Excl. active sharing)','Annual Maintenance Fee ORANGE (Excl. active sharing)','Other Services Fee ORANGE (Excl. active sharing)',\
             'Total Revenues ORANGE(Excl. active sharing)','Total Revenues ORANGE(Excl. active sharing) (with Discount)','Residual duration ORANGE(Excl. active sharing)',\
             'Maturity Clusters ORANGE(Excl. active sharing)','RCS&RDS','Annual Hosting RCS&RDS','Annual Hosting RCS&RDS (with Discount)','Annual Energy RCS&RDS',\
             'Annual Maintenance Fee RCS&RDS','Other Services Fee RCS&RDS','Total Revenues RCS&RDS','Total Revenues RCS&RDS (with Discount)','Residual duration RCS&RDS',\
             'Maturity Clusters RCS&RDS','OTMO','Annual Hosting OTMOs','Annual Hosting OTMOs (with Discount)','Annual Energy OTMOs','Annual Maintenance Fee OTMOs',\
             'Other Services Fee OTMOs','Total Revenues OTMOs','Total Revenues OTMOs (with Discount)','Residual duration OTMOs','Maturity Clusters OTMOs',\
             'Total # of 3rd Party Tenants (excluding active sharing)','Annual Fee from 3rd Party Tenants','Annual Fee from 3rd Party Tenants (with Discount)',\
             'Annual Energy Fee from 3rd Party Tenants','Annual Maintenance Fee from 3rd Party Tenants','Other Services Fee from 3rd Party Tenants',\
             'Total Hosting Fee & Services from 3rd Party Tenants','Total Hosting Fee & Services from 3rd Party Tenants (with Discount)',\
             'Waighted Average residual duration','Macro Cluster Tenancy','Final Cluster Type of Contract','Macro Cluster 1','Sites w/ at list a DDS (Lease Contract Type)',\
             '# of Tenants','Categorization by Tenant combination','Categorization by Type of Passive contracts','Type of contract (principle contract)',\
             'Cluster Type of contract (principle contract)','Easement (Servitù di passaggio)','Final Cluster','Turistic sites','TLK_1','Orange','RCS&RDS_1','OTMOs',\
             'Orange Active Shared Sites ','Annual Hosting ORANGE','No_3','Annual Energy ORANGE','No_4','Total Revenues ORANGE','Only - Active Sharing Orange',\
             'Total # of 3rd Party Tenants (including active sharing)','Total 3rd Party Tenants hosting revenues (including active sharing)','FSO Criticality',\
             'TX LH POC','10% Critical','EVO SAP ID ','Enhanced BBU','Diameter\nVodafone Antenna','Diameter\n(Orange rural & Unilateranl ',\
             'Total diameter of microwave and mmWave antennas','Is the Site Under Construction','indicative completion date',\
             'Capital expenditure incurred to the MSA Effective Date','indicative capital expenditure required to complete the site build',\
             'proposed configuration for the completed site',\
             'Details of the Operator Equipment installed at the Site (including details of the Standard Configuration Attributes set out in Schedule 7 (Standard Configuration)',\
             'Details of the access arrangements applicable at each Site (including any access restrictions and applicable public access requirements)',\
             'Is the Existing Configuration is within the applicable Standard Configuration?','Bundled sites','Strategic Sites','Unilateral Orange Transmission Site',\
             'First 10 Unilateral Orange Transmission Site','Details of any Orange Equipment at Orange Shared Sites ','Transfer_Date_Of_Registration_Required_Sites',\
             'Wip_Site','Bts_Site','Sites_As_Metered_Estimated','Strategic_Site_Bucket','Subsequent_Sharing_Arrangement','First_Active_Sharing_Start_Date',\
             'First_Active_Sharing_End_Date','Sites_Within_500_Macro_Sites','Date_Of_Equipment_Removal','RFAI ( Ready For Active Installation )','X','Site Status',\
             'Critical site']

"""Defining variables which is gonna be reusable in checks"""
tw_index = 'Code'
tw_doer = 'Date_Of_Equipment_Removal'
tw_status = 'Site Status'
tw_bts = 'Bts_Site'
tw_bill = 'Billing Trigger Date'
tw_wip = 'Wip_Site'
tw_decom = 'Decommissioned sites'
tw_critical = 'Critical site'

path_msa = '/content/TowerDB_Romania_20210630.csv'
msa = pd.read_csv(path_msa, encoding='latin')
msa.columns = col_order
msa_index = 'Code'
msa_doer = 'Date_Of_Equipment_Removal'
msa_status = 'Site Status'
msa_bts = 'Bts_Site'
msa_bill = 'Billing Trigger Date'
msa_wip = 'Wip_Site'
msa_decom = 'Decommissioned sites'
msa_critical = 'Critical site'

In [11]:
towerdb.rename({'VDF ENERGY COST EUR': 'Energy cost FYTD_DEC_20', 'VDF ENERGY Consum kWh': 'Energy consumption FYTD_DEC_20'}, axis=1, inplace=True)
towerdb = towerdb.reindex(columns=col_order)

In [ ]:
"""Check Columns Received"""             
df_cols = check_columns_received(towerdb, col_order)
#No columns missing
df_cols

First Check - Dates Formats (dd/mm/YYYY)

Columns: Date of equipment removal (from MAR´21)

In [ ]:
"""You need to convert all values in cols for string format to check"""
# Columns to functions
dates_doer = [tw_index, tw_doer]
#Columns to parser
bill=[tw_bill]
doer=[tw_doer]

#actives_1 = towerdb[towerdb[tw_status]=='In Service']
no_actives_1 = towerdb[towerdb[tw_status]=='Dismantled']

date_parser(towerdb, bill, "%d/%m/%Y", 'no')
date_parser(no_actives_1, doer, "%d/%m/%Y", 'mixed')


#Checking columns for errors
# actives_dates_errors = check_date_columns(actives_1, tw_index, dates_bill, 2) 
# Actives sites with blank billing trigger date
no_actives_dates_errors = check_date_columns(no_actives_1, tw_index, dates_doer, 2) 
no_actives_dates_errors

Second Check - TW Amount value General (xxx.xx)

Column(s): ???

In [ ]:
amount_cols = [tw_index, """???"""]
df_amount_errors = check_amounts(actives, tw_index, amount_cols, 1)
#No one error

Thirth - Check Picklist values All sites

Do this check in all sites

Check the picklist for each case

In [ ]:
picklist_tw_general = {
    'Categorization by Transmission Sys': ['0','Long-term Mobile', 'Macro', 'Outdoor Small Cells', 'Public DAS',\
                                           'Repeater', 'Transmission', 'w/o equipment']
}
pick_col_general = ['Code', 'Categorization by Transmission Sys']

df_general_pick = check_picklist(towerdb, tw_index, pick_col_general, picklist_tw_general)
df_general_pick
#no errors

Fourth Check - Remove "N/A", "0" or "-" values

In [ ]:
towerdb = replace_values(towerdb, col_order)

Fifth Check MoM Sites (BTS, decomissoned...)

In [61]:
""" BTS sites"""
path_uip = '/content/UserInput_Romania.xlsx'
uip_names = ['Site_ID','Site Categorization', 'BTS site applicable charge (Annual)',\
             'Commercials for sites beyond 10% cap of critical sites (Annual)']
uip = pd.read_excel(path_uip ,sheet_name='SiteLevel',usecols=[0,1,2,3],skiprows=2)
uip.columns = uip_names

msa_sites = [str(i) for i in msa[msa_index]]
tw_sites = [str(i) for i in towerdb[tw_index]]
uip_sites = [str(i) for i in uip['Site_ID']]

In [4]:
actives = towerdb[towerdb[tw_status]=='In Service']
df_mom_bts = check_mom_bts(actives, tw_index, tw_bts, msa, msa_index, msa_bts)
# No one error df_mom_bts

decomiss = towerdb[towerdb[tw_decom]=='Yes']
df_mom_decom = check_mom_bts(decomiss, tw_index, tw_decom, msa, msa_index, msa_decom)
#No errors

Check Picklist and dates formats for In service sites

In [ ]:
picklis_dict = {
    'Categorization by Transmission Sys': ['Long-term Mobile', 'Macro', 'Outdoor Small Cells', 'Public DAS',\
                                           'Repeater', 'Transmission', 'w/o equipment'],
    'Categorization by Site Type': ['DAS passive','GBT','RTT', 'Outdoor Small Cells'],
    'Sites_As_Metered_Estimated': ['Estimated Model','Metered Model'],
    'Infrastructure ready (existing)/ to be ready (new)': ['Yes', 'No'],
    'Air Conditioning': ['No','Yes'],
    'Bts_Site': ['Yes', 'No'],
    'Strategic Sites': ['Yes', 'No'],
    'Strategic_Site_Bucket': ['Non Strategic'],
    'Critical site': ['Yes', 'No'],
    '10% Critical': ['Within 10%','Non Critical'],
    'Wip_Site': ['Yes', 'No'],
    'Bundled sites': ['Yes', 'No'],
    'Decommissioned sites': ['Yes', 'No']
}

picklist_cols = ['Code','Categorization by Transmission Sys','Categorization by Site Type','Sites_As_Metered_Estimated',\
                'Infrastructure ready (existing)/ to be ready (new)','Air Conditioning','Bts_Site','Strategic Sites',\
                'Strategic_Site_Bucket','Critical site','10% Critical','Wip_Site','Bundled sites','Decommissioned sites']

df_in_service_picklist = check_picklist(actives, tw_index, picklist_cols, picklis_dict)
#Tem error em 3 colunas 

In [ ]:
#check dates in columns
start_dates = ['First_Active_Sharing_Start_Date', 'First_Active_Sharing_End_Date']

date_parser(actives, start_dates, "%d/%m/%Y", 'no')
#actives['First_Active_Sharing_End_Date'] = actives['First_Active_Sharing_End_Date'].fillna('')
in_service_cols = ['Code', 'First_Active_Sharing_Start_Date', 'First_Active_Sharing_End_Date']
df_in_service_dates = check_date_columns(actives, tw_index, in_service_cols, 2)
df_in_service_dates
# Varias linhas em branco

Fifth Check BTS Flagged(Billing Trigger and Commercial)

In [ ]:
actives_1 = towerdb[towerdb[tw_status]=='In Service']
status = 'Yes'
df_bts_flagged = check_tw_bill_doer(actives_1, tw_index,tw_bill, tw_bts, status, 'bill')


Usar as datas no formato Datetime para rodar esse check

In [ ]:
#actives_1 = towerdb[towerdb[tw_status]=='In Service']
new_sites, bts_out_uip, df_bts_errors = check_new_sites(towerdb, tw_index, tw_bts, tw_bill, msa_sites, tw_sites, uip_sites)

Check WIP Flagged sites


In [32]:
# Pode ter errors nos Codes por ficarem em int no momento que roda
wip_out_tw_list, df_wip_and_bts_flagged = check_wip(towerdb,tw_index, tw_wip, tw_bts, msa, msa_index, msa_wip)
# No errors

Check Decomissioned sites

In [ ]:
df_decom_sites = check_decommissioned(towerdb, tw_index, tw_decom, tw_doer)
df_decom_sites
#No errors

Check Doer columns for in service sites

Should not to be in past or different of blank

In [ ]:
#Coluna Doer tem valores fora do formato
actives = towerdb[towerdb[tw_status]=='In Service']
df_doer = check_tw_bill_doer(actives, tw_index, tw_doer, tw_bts, 'Yes', 'doer')
df_doer
#No errors

BTS sites are in subsequent Month

In [ ]:
#Retorna os sites novos
df_bts_out = check_bts(towerdb, tw_bts, tw_index, msa, msa_bts, msa_index)
# No errors

*Tenth* - Check UIP Towerdb matches

In [62]:
in_service_uip_sites, decomiss_sites_in_uip, bts_sites_out_uip, critical = check_uip_tw(towerdb,tw_index, tw_status, \
                                                              tw_decom, tw_bts,tw_critical, \
                                                              uip, uip_sites, 'ro')

Commercial

In [ ]:

path_before = '/content/UserInput_Romania_20210630.xlsx'
names = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2','Data_Type','Input_Value_1','Input_Value_2',\
        'Description/Instruction', 'Frequency of Update']
merge_cols = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2', 'Data_Type', \
              'Description/Instruction', 'Frequency of Update']
cols_ordered = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2','Data_Type','Input_Value_1_actual',\
                'Input_Value_1_before','Input_Value_2_actual','Input_Value_2_before','Equal Values',\
                'Description/Instruction', 'Frequency of Update']
replace_values=['Input_Value_1','Input_Value_2']
df_commercial_diffs = check_commercial(path_uip, path_before, replace_values, names, merge_cols, cols_ordered)
df_commercial_diffs
#Erro na Coluna Param2 Vazio no anterior preenchido no atual

Creating Excel Log

In [121]:
df_list = [df_in_service_picklist, df_in_service_dates]
sheetnames = ['Sites with Picklist Error', 'Incorrect date format(In Service)']

path = '/content/towerdb_ro_errors.xlsx'
general_log_erros(df_list, sheetnames, path)

/usr/local/lib/python3.7/dist-packages/openpyxl/workbook/child.py:102: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


TA Input

In [ ]:
pathtw = '/content/TowerDB_Romania_20210731.xlsx'
sheet= 'Tenant Template'
skipr = 7
skipc = 2
ta = read_files(pathtw, sheet, skipr, skipc, tw_index)

ta_cols = ['Code', 'Current Annual Fee per Tenant (Annual Hosting (with Discount))']  

#ta['Importe anual'] = ta['Importe anual'].astype(str)

df_ta_amount = check_amounts(ta, 'Code', ta_cols, 1)
# No errors

df_ta_dates = check_lc_ta_dates(ta,'Code', 'Starting date', 'Expiring date')
#NO erros